In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#importing required libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
df=pd.read_csv("/content/drive/My Drive/finaldataset.csv")  # Loading the dataset to a dataframe
df.head()

,Unnamed: 0,Date,subjectivity,pos,neg,neu,compound,Open,High,Low,Close,Volume,Adj Close,change in percentage (%),Label
0,0,2008-08-08,0.15484,0.08800,0.24204,0.66984,-0.309516,11432.089844,11759.959961,11388.040039,11734.320312,212830000.0,11734.320312,2.643703,1
1,1,2008-08-11,0.16664,0.10640,0.26608,0.62756,-0.170236,11729.669922,11867.110352,11675.530273,11782.349609,183190000.0,11782.349609,0.449115,1
2,2,2008-08-12,0.34480,0.10252,0.21792,0.67952,-0.207112,11781.700195,11782.349609,11601.519531,11642.469727,173590000.0,11642.469727,-1.181752,0
3,3,2008-08-13,0.15584,0.10040,0.16168,0.73792,-0.103452,11632.809570,11633.780273,11453.339844,11532.959961,182550000.0,11532.959961,-0.858345,0
4,4,2008-08-14,0.27080,0.12120,0.22724,0.65160,-0.166048,11532.070312,11718.280273,11450.889648,11615.929688,159790000.0,11615.929688,0.727184,1


In [4]:
dates=df['Date'] #saving dates feature 

In [5]:
news=pd.read_csv("/content/drive/My Drive/RedditNews.csv") # Loading the dataset to a dataframe
news.head()

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,"The president of France says if Brexit won, so..."
3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...


In [6]:
news=news.reset_index() #reseting the index

In [7]:
news = news[::-1].reset_index(drop=True) #reverseing the indexing of the dataframe
   

In [8]:
news['Date'] = pd.to_datetime(news['Date']) #changing the datatype of date feature

In [9]:
price = []
headlines = []

for row in df.iterrows():
    daily_headlines = []
    date = row[1]['Date']
    price.append(row[1]['Open'])
    for row_ in news[news.Date==date].iterrows(): #collecting all the rows having same dates into a list
        daily_headlines.append(row_[1]['News'])
    
    # Track progress
    headlines.append(daily_headlines)
    if len(price) % 500 == 0:
        print(len(price))

500
1000
1500


In [10]:
headlines[0]

['b"No Help for Mexico\'s Kidnapping Surge"',
 "b'Visitors Suffering from Mental Illnesses Banned from Olympics'",
 'b\'Indian shoe manufactory  - And again in a series of "you do not like your work?"\'',
 "b'Caucasus in crisis: Georgia invades South Ossetia'",
 "b'Why the Pentagon Thinks Attacking Iran is a Bad Idea - US News &amp; World Report'",
 'b"Georgia will withdraw 1,000 soldiers from Iraq to help fight off Russian forces in Georgia\'s breakaway region of South Ossetia"',
 "b'This is a busy day:  The European Union has approved new sanctions against Iran in protest at its nuclear programme.'",
 'b\'Condoleezza Rice: "The US would not act to prevent an Israeli strike on Iran." Israeli Defense Minister Ehud Barak: "Israel is prepared for uncompromising victory in the case of military hostilities."\'',
 "b'Al-Qaeda Faces Islamist Backlash'",
 "b'Georgia Invades South Ossetia - if Russia gets involved, will NATO absorb Georgia and unleash a full scale war?'",
 "b'Did World War III

In [12]:
df2=pd.DataFrame(headlines) #converting a list to dataframe

In [13]:
#renaming the column names 
list1= [i for i in range(25)]
new_Index=[str(i) for i in list1]
df2.columns= new_Index
df2.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,"b""No Help for Mexico's Kidnapping Surge""",b'Visitors Suffering from Mental Illnesses Ban...,b'Indian shoe manufactory - And again in a se...,b'Caucasus in crisis: Georgia invades South Os...,b'Why the Pentagon Thinks Attacking Iran is a ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'This is a busy day: The European Union has ...,"b'Condoleezza Rice: ""The US would not act to p...",b'Al-Qaeda Faces Islamist Backlash',b'Georgia Invades South Ossetia - if Russia ge...,b'Did World War III start today?',"b""China tells Bush to stay out of other countr...","b""So---Russia and Georgia are at war and the N...",b'Announcing:Class Action Lawsuit on Behalf of...,b'Rice Gives Green Light for Israel to Attack ...,b'Did the U.S. Prep Georgia for War with Russia?',b'Georgian troops retreat from S. Osettain cap...,"b""The 'enemy combatent' trials are nothing but...","b""Breaking: Georgia invades South Ossetia, Rus...",b'150 Russian tanks have entered South Ossetia...,"b""Afghan children raped with 'impunity,' U.N. ...",b'Russian tanks are moving towards the capital...,b'Russia Today: Columns of troops roll into So...,b'BREAKING: Musharraf to be impeached.',"b""Georgia 'downs two Russian warplanes' as cou..."
1,"b""So this is what it's come to: trading sex fo...",b'Russia is so much better at war',b'Perhaps *the* question about the Georgia - R...,b' Russia has just beaten the United States ov...,b'Israeli Physicians Group Condemns State Tort...,b'War in South Ossetia [PICS]',b'China to overtake US as largest manufacturer',b'Riots are still going on in Montreal (Canada...,"b'""Do not believe TV, neither Russian nor Geor...",b'Israel and the US behind the Georgian aggres...,b'The French Team is Stunned by Phelps and the...,b'Drivers in a Jerusalem taxi station threaten...,b' U.S. ship heads for Arctic to define territ...,b'Abhinav Bindra wins first ever Individual Ol...,b'Russia presses deeper into Georgia; U.S. say...,"b""Georgia's move, a mistake of monumental prop...",b'Welcome To World War IV! Now In High Definit...,b'An American citizen living in S.Ossetia blam...,b'Russia angered by Israeli military sale to G...,b'What were the Mossad with fraudulent New Zea...,"b""Olympic opening ceremony fireworks 'faked'""",b'Georgian army flees in disarray as Russians ...,"b""Jewish Georgian minister: Thanks to Israeli ...",b'Bush puts foot down on Georgian conflict',b'Why wont America and Nato help us? If they w...
2,"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",b'America: The New Mexico?',b'Christopher King argues that the US and NATO...,b'All signs point to the US encouraging Georgi...,b'War in Georgia: The Israeli connection',b'Remember that adorable 62-year-old who led y...,"b'Russia, Georgia, and NATO: Cold War Two'","b'Gorbachev accuses U.S. of making a ""serious ...",b'Why Russias response to Georgia was right',b'U.S. troops still in Georgia (did you know t...,b'The 11 Top Party Cities in the World',b'55 pyramids as large as the Luxor stacked in...,b'Beginning a war as the Olympics were opening...,b'CNN use footage of Tskhinvali ruins to cover...,"b'Gorbachev: ""Georgian military attacked the S...",b'U.S. Beats War Drum as Iran Dumps the Dollar',"b""The US military was surprised by the timing ...","b""I'm Trying to Get a Sense of This Whole Geor...",b'Stratfor: The Russo-Georgian War and the Bal...,b'Why Microsoft and Intel tried to kill the XO...,b'Ceasefire in Georgia: Putin Outmaneuvers the...,"b""Al-Qa'eda is losing support in Iraq because ...","b'""If we had no sexual harassment we would hav...","b""Russia 'ends Georgia operation'""",b'Remember that adorable 9-year-old who sang a...
3,b'2006: Nobel laureate Aleksander Solzhenitsyn...,b'Georgian president says US military will ta...,b' Quarter of Russians blame U.S. for conflict...,b'Witness: Russian forces head towards Tbilisi...,b'Gorbachev: We Had No Choice',b'Israeli defence minister: US against st

In [14]:
#merging the dates to df2n
merged = pd.concat([dates, df2], axis=1, sort=False)
merged.head()

,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,2008-08-08,"b""No Help for Mexico's Kidnapping Surge""",b'Visitors Suffering from Mental Illnesses Ban...,b'Indian shoe manufactory - And again in a se...,b'Caucasus in crisis: Georgia invades South Os...,b'Why the Pentagon Thinks Attacking Iran is a ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'This is a busy day: The European Union has ...,"b'Condoleezza Rice: ""The US would not act to p...",b'Al-Qaeda Faces Islamist Backlash',b'Georgia Invades South Ossetia - if Russia ge...,b'Did World War III start today?',"b""China tells Bush to stay out of other countr...","b""So---Russia and Georgia are at war and the N...",b'Announcing:Class Action Lawsuit on Behalf of...,b'Rice Gives Green Light for Israel to Attack ...,b'Did the U.S. Prep Georgia for War with Russia?',b'Georgian troops retreat from S. Osettain cap...,"b""The 'enemy combatent' trials are nothing but...","b""Breaking: Georgia invades South Ossetia, Rus...",b'150 Russian tanks have entered South Ossetia...,"b""Afghan children raped with 'impunity,' U.N. ...",b'Russian tanks are moving towards the capital...,b'Russia Today: Columns of troops roll into So...,b'BREAKING: Musharraf to be impeached.',"b""Georgia 'downs two Russian warplanes' as cou..."
1,2008-08-11,"b""So this is what it's come to: trading sex fo...",b'Russia is so much better at war',b'Perhaps *the* question about the Georgia - R...,b' Russia has just beaten the United States ov...,b'Israeli Physicians Group Condemns State Tort...,b'War in South Ossetia [PICS]',b'China to overtake US as largest manufacturer',b'Riots are still going on in Montreal (Canada...,"b'""Do not believe TV, neither Russian nor Geor...",b'Israel and the US behind the Georgian aggres...,b'The French Team is Stunned by Phelps and the...,b'Drivers in a Jerusalem taxi station threaten...,b' U.S. ship heads for Arctic to define territ...,b'Abhinav Bindra wins first ever Individual Ol...,b'Russia presses deeper into Georgia; U.S. say...,"b""Georgia's move, a mistake of monumental prop...",b'Welcome To World War IV! Now In High Definit...,b'An American citizen living in S.Ossetia blam...,b'Russia angered by Israeli military sale to G...,b'What were the Mossad with fraudulent New Zea...,"b""Olympic opening ceremony fireworks 'faked'""",b'Georgian army flees in disarray as Russians ...,"b""Jewish Georgian minister: Thanks to Israeli ...",b'Bush puts foot down on Georgian conflict',b'Why wont America and Nato help us? If they w...
2,2008-08-12,"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",b'America: The New Mexico?',b'Christopher King argues that the US and NATO...,b'All signs point to the US encouraging Georgi...,b'War in Georgia: The Israeli connection',b'Remember that adorable 62-year-old who led y...,"b'Russia, Georgia, and NATO: Cold War Two'","b'Gorbachev accuses U.S. of making a ""serious ...",b'Why Russias response to Georgia was right',b'U.S. troops still in Georgia (did you know t...,b'The 11 Top Party Cities in the World',b'55 pyramids as large as the Luxor stacked in...,b'Beginning a war as the Olympics were opening...,b'CNN use footage of Tskhinvali ruins to cover...,"b'Gorbachev: ""Georgian military attacked the S...",b'U.S. Beats War Drum as Iran Dumps the Dollar',"b""The US military was surprised by the timing ...","b""I'm Trying to Get a Sense of This Whole Geor...",b'Stratfor: The Russo-Georgian War and the Bal...,b'Why Microsoft and Intel tried to kill the XO...,b'Ceasefire in Georgia: Putin Outmaneuvers the...,"b""Al-Qa'eda is losing support in Iraq because ...","b'""If we had no sexual harassment we would hav...","b""Russia 'ends Georgia operation'""",b'Remember that adorable 9-year-old who sang a...
3,2008-08-13,b'2006: Nobel laureate Aleksander Solzhenitsyn...,b'Georgian president says US military will ta...,b' Quarter of Russians blame U.S. for conflict...,b'Witness: Russian forces head towards Tbilisi...,b'Gorbachev: We Had No 

In [15]:
df3=df['Label']

In [17]:
#merging label feature to dtaframe for furthur prediction process
merged=pd.concat([merged,df3],axis=1)
merged.head()

,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Label,Label
0,2008-08-08,"b""No Help for Mexico's Kidnapping Surge""",b'Visitors Suffering from Mental Illnesses Ban...,b'Indian shoe manufactory - And again in a se...,b'Caucasus in crisis: Georgia invades South Os...,b'Why the Pentagon Thinks Attacking Iran is a ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'This is a busy day: The European Union has ...,"b'Condoleezza Rice: ""The US would not act to p...",b'Al-Qaeda Faces Islamist Backlash',b'Georgia Invades South Ossetia - if Russia ge...,b'Did World War III start today?',"b""China tells Bush to stay out of other countr...","b""So---Russia and Georgia are at war and the N...",b'Announcing:Class Action Lawsuit on Behalf of...,b'Rice Gives Green Light for Israel to Attack ...,b'Did the U.S. Prep Georgia for War with Russia?',b'Georgian troops retreat from S. Osettain cap...,"b""The 'enemy combatent' trials are nothing but...","b""Breaking: Georgia invades South Ossetia, Rus...",b'150 Russian tanks have entered South Ossetia...,"b""Afghan children raped with 'impunity,' U.N. ...",b'Russian tanks are moving towards the capital...,b'Russia Today: Columns of troops roll into So...,b'BREAKING: Musharraf to be impeached.',"b""Georgia 'downs two Russian warplanes' as cou...",1,1
1,2008-08-11,"b""So this is what it's come to: trading sex fo...",b'Russia is so much better at war',b'Perhaps *the* question about the Georgia - R...,b' Russia has just beaten the United States ov...,b'Israeli Physicians Group Condemns State Tort...,b'War in South Ossetia [PICS]',b'China to overtake US as largest manufacturer',b'Riots are still going on in Montreal (Canada...,"b'""Do not believe TV, neither Russian nor Geor...",b'Israel and the US behind the Georgian aggres...,b'The French Team is Stunned by Phelps and the...,b'Drivers in a Jerusalem taxi station threaten...,b' U.S. ship heads for Arctic to define territ...,b'Abhinav Bindra wins first ever Individual Ol...,b'Russia presses deeper into Georgia; U.S. say...,"b""Georgia's move, a mistake of monumental prop...",b'Welcome To World War IV! Now In High Definit...,b'An American citizen living in S.Ossetia blam...,b'Russia angered by Israeli military sale to G...,b'What were the Mossad with fraudulent New Zea...,"b""Olympic opening ceremony fireworks 'faked'""",b'Georgian army flees in disarray as Russians ...,"b""Jewish Georgian minister: Thanks to Israeli ...",b'Bush puts foot down on Georgian conflict',b'Why wont America and Nato help us? If they w...,1,1
2,2008-08-12,"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",b'America: The New Mexico?',b'Christopher King argues that the US and NATO...,b'All signs point to the US encouraging Georgi...,b'War in Georgia: The Israeli connection',b'Remember that adorable 62-year-old who led y...,"b'Russia, Georgia, and NATO: Cold War Two'","b'Gorbachev accuses U.S. of making a ""serious ...",b'Why Russias response to Georgia was right',b'U.S. troops still in Georgia (did you know t...,b'The 11 Top Party Cities in the World',b'55 pyramids as large as the Luxor stacked in...,b'Beginning a war as the Olympics were opening...,b'CNN use footage of Tskhinvali ruins to cover...,"b'Gorbachev: ""Georgian military attacked the S...",b'U.S. Beats War Drum as Iran Dumps the Dollar',"b""The US military was surprised by the timing ...","b""I'm Trying to Get a Sense of This Whole Geor...",b'Stratfor: The Russo-Georgian War and the Bal...,b'Why Microsoft and Intel tried to kill the XO...,b'Ceasefire in Georgia: Putin Outmaneuvers the...,"b""Al-Qa'eda is losing support in Iraq because ...","b'""If we had no sexual harassment we would hav...","b""Russia 'ends Georgia operation'""",b'Remember that adorable 9-year-old who sang a...,0,0
3,2008-08-13,b'2006: Nobel laureate Aleksander Solzhenitsyn...,b'Georgian president says US military will ta...,b' Quarter of Russians blame U.S. for conflict...,b'Witness: Russian forces head towards Tbilisi...

In [ ]:
#splitting of data into train and test
train = merged[merged['Date'] < '20150101']
test = merged[merged['Date'] > '20141231']

In [ ]:
#removing special characters
slicedData= train.iloc[:,1:26]
slicedData.replace(to_replace="[^a-zA-Z]", value=" ", regex=True, inplace=True)

# Convertng headlines to lower case
for index in new_Index:
    slicedData[index]=slicedData[index].astype(str).str.lower()
slicedData.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,ozone layer hole seems to be healing us amp...,a hindu temple worker has been killed by three...,venezuela where anger over food shortages is ...,spain arrests three pakistanis accused of prom...,philippines president rodrigo duterte urges pe...,mass fish kill in vietnam solved as taiwan ste...,u s sailors detained by iran spoke too much u...,india gets billion loan from world bank for...,s amp p revises european union credit rating t...,the united states has placed myanmar uzbekist...,u n panel moves to curb bias against l g b t ...,iran celebrates american human rights week mo...,taiwanese warship accidentally fires missile t...,abbas plo faction calls killer of year old ...,france cracks down on factory farms a viral ...,china kills millions of innocent meditators fo...,switzerland denies muslim girls citizenship af...,facebook wins privacy case can track any belg...,austria s highest court annuls presidential el...,brazil huge spike in number of police killing...,nobel laureates urge greenpeace to stop o...,british man who must give police hours not...,the president of france says if brexit won so...,imf chief backs athens as permanent olympic host,a year old woman in mexico city finally re...


In [ ]:
headlines = []
for row in range(0,len(slicedData.index)):
    headlines.append(' '.join(str(x) for x in slicedData.iloc[row,0:25]))

Logistic Regresson model using count vectorization and parameter n gram

In [ ]:
#ngram (1,1)

In [ ]:
basicvectorizer = CountVectorizer(ngram_range=(1,1))
basictrain = basicvectorizer.fit_transform(headlines)
basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(basictrain, train["Label"]) 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:26]))
basictest = basicvectorizer.transform(testheadlines)
predictions = basicmodel.predict(basictest)

In [ ]:
print(basictrain.shape)

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

print (classification_report(test["Label"], predictions))
print (accuracy_score(test["Label"], predictions))

(1863, 32475)
              precision    recall  f1-score   support

           0       0.82      0.85      0.84       183
           1       0.86      0.83      0.84       195

    accuracy                           0.84       378
   macro avg       0.84      0.84      0.84       378
weighted avg       0.84      0.84      0.84       378

0.8386243386243386


In [ ]:
#ngram (2,2)

In [ ]:
basictest2 = basicvectorizer2.transform(testheadlines)
predictions2 = basicmodel2.predict(basictest2)
basicmodel2 = LogisticRegression()
basicmodel2 = basicmodel2.fit(basictrain2, train["Label"])
basicmodel2 = LogisticRegression()
basicmodel2 = basicmodel2.fit(basictrain2, train["Label"])
print(basictrain2.shape)
print (classification_report(test["Label"], predictions2))
print (accuracy_score(test["Label"], predictions2))

(1863, 409523)
              precision    recall  f1-score   support

           0       0.91      0.80      0.85       183
           1       0.83      0.93      0.88       195

    accuracy                           0.87       378
   macro avg       0.87      0.87      0.87       378
weighted avg       0.87      0.87      0.87       378

0.8677248677248677


In [ ]:
#ngram (3,3)

In [ ]:
basicvectorizer3 = CountVectorizer(ngram_range=(3,3))
basictrain3 = basicvectorizer3.fit_transform(headlines)
print(basictrain3.shape)

basicmodel3 = LogisticRegression()
basicmodel3 = basicmodel3.fit(basictrain3, train["Label"])

basictest3 = basicvectorizer3.transform(testheadlines)
predictions3 = basicmodel3.predict(basictest3)

print (classification_report(test["Label"], predictions3))
print (accuracy_score(test["Label"], predictions3))

(1863, 702474)
              precision    recall  f1-score   support

           0       0.99      0.73      0.84       183
           1       0.79      0.99      0.88       195

    accuracy                           0.86       378
   macro avg       0.89      0.86      0.86       378
weighted avg       0.89      0.86      0.86       378

0.8624338624338624
